In [1]:
import  numpy as np

In [2]:
def Sigmoide(x):
    return 1/(1+np.exp(-x))


def my_dense(a_in, W, b):
    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):
        w = W[:,j]
        z = np.dot(w,a_in)+ b[j]
        a_out[j] = Sigmoide(z)
    return a_out

def my_Sequential(x, W1, b1, W2, b2):
    a1 = my_dense(x, W1, b1)
    a2 = my_dense(a1, W2, b2)
    return a2

def my_predict(X, W1, b1, W2, b2):
    N = X.shape[0]
    y = np.zeros((N, 1))  # Initialize y as a 2D array
    for i in range(N):
        y[i] = my_Sequential(X[i], W1, b1, W2, b2)
    return y

Now let's test our functions

In [3]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

In [4]:
import tensorflow as tf

# Create the normalization layer
norm_l = tf.keras.layers.Normalization(axis=-1)

# Adapt the normalizer to the given ranges
data = np.array([
    [284.99, 15.45],
    [151.32, 11.51]
])
norm_l.adapt(data)

In [5]:
X_tst = np.array([
    [200,13.9],  
    [200,17]])   
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)

In [6]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


In [7]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1 
    else:       
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


In [8]:
# Vectorized form of my_dense which is better
def my_dense_vectorized(a_in, W, b):
    Z = np.matmul(a_in,W)+b
    A_out = Sigmoide(Z)
    return A_out
    

Note that in vectorized form X should be 2D array and also b so that we can make matrix mathimatical operations on them and this will be more efficient

In [9]:
# softmax function for multiclass classification

def my_soft_max(Z):
    expZ = np.exp(Z)
    A_out = expZ / np.sum(expZ, axis=1, keepdims=True)
    return A_out

now we will write two codes for multicalss calssification and we will dicover which is better

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.datasets import make_blobs

In [12]:
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

this works but not the better version

In [20]:
model = Sequential([
    Dense(25, activation='relu'),
    Dense(15, activation='relu'),
    Dense(4, activation='softmax')
])
model.compile( 
    loss=SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
 )
model.fit(X_train, y_train, epochs=10)


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.9114
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4212
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1874
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0999
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0838
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0669
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0549
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0463
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0435
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0395


In [21]:
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred [:2])
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[8.6346976e-03 2.9250854e-03 9.6341825e-01 2.5021996e-02]
 [9.9623054e-01 3.6752594e-03 8.7207896e-05 7.0505980e-06]]
largest value 0.9999999 smallest value 8.2444124e-10


now let's go to the better implementation

In [22]:
preferred_model = Sequential([
    Dense(25, activation='relu'),
    Dense(15, activation='relu'),
    Dense(4, activation='linear')
])
preferred_model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001)
 )
preferred_model.fit(
    X_train,y_train,
    epochs=10
)


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3101
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4594
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - loss: 0.2224
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1311
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0872
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0730
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - loss: 0.0575
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0548
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0527
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0523


In [23]:
p_preferred = preferred_model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
two example output vectors:
 [[-2.0152242  -3.1272018   2.1945581  -1.6836379 ]
 [ 4.9682794   0.36170593 -5.3457966  -8.002313  ]]
largest value 8.179369 smallest value -12.3425255


we must handle the output

In [24]:
sm_preferred = tf.nn.softmax(preferred_model.predict(X_train))
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
two example output vectors:
 [[1.4272669e-02 4.6943934e-03 9.6114862e-01 1.9884354e-02]
 [9.9007797e-01 9.8868944e-03 3.2833963e-05 2.3046907e-06]]
largest value 0.9999951 smallest value 3.7562935e-09


In [25]:
for i in range(5):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

[-2.0152242 -3.1272018  2.1945581 -1.6836379], category: 2
[ 4.9682794   0.36170593 -5.3457966  -8.002313  ], category: 0
[ 3.49983     0.62880266 -4.08026    -6.347795  ], category: 0
[-1.1976029  3.680969  -2.5802124 -2.531852 ], category: 1
[-1.5353869 -5.1511602  3.5123003 -5.13287  ], category: 2


let's dive in a multicalss classification problem with 4 calsses

In [26]:
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(n_samples=m, centers=centers, cluster_std=std,random_state=30)

In [29]:
# show classes in data set
print(f"unique classes {np.unique(y_train)}")
# show how classes are represented
print(f"class representation {y_train[:10]}")
# show shapes of our dataset
print(f"shape of X_train: {X_train.shape}, shape of y_train: {y_train.shape}")

unique classes [0 1 2 3]
class representation [3 3 3 0 3 3 3 3 2 0]
shape of X_train: (100, 2), shape of y_train: (100,)


now let's build our model

In [33]:
tf.random.set_seed(1234)
model_1 = Sequential([
    Dense(2, activation='relu' , name ="L1"),
    Dense(4, activation='linear', name ="L2")
])

In [34]:
model_1.compile(
    loss = SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

model_1.fit(X_train,y_train,epochs=200)

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1312  
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9906 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - loss: 0.9167
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8757 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8489 
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8256 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - loss: 0.8022
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7784 
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7548 
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7322 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7111 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6917 
Epoch 13/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6736 
Epoch 14/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6567 
Epoch 15/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6407 
Epoch 16/200
4/

let's dive deeper to understand how the dicesion boundries has been generated

In [35]:
# gather the trained parameters from the first layer
l1 = model_1.get_layer("L1")
W1,b1 = l1.get_weights()